In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import desc
from pyspark.sql.functions import countDistinct

In [2]:
spark_avia = SparkSession.builder.master("yarn").appName("avia").config("spark.jars", "/home/teran45/spark/postgresql-42.3.4.jar").getOrCreate()

2022-05-26 02:41:32,250 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-26 02:41:33,910 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
df = spark_avia.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/dataengineering") \
    .option("dbtable", "public.opensky") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [4]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- icao24: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- time_position: integer (nullable = true)
 |-- last_contact: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- baro_altitude: double (nullable = true)
 |-- on_ground: boolean (nullable = true)
 |-- velocity: double (nullable = true)
 |-- true_track: double (nullable = true)
 |-- vertical_rate: double (nullable = true)
 |-- sensors: integer (nullable = true)
 |-- geo_altitude: double (nullable = true)
 |-- squawk: string (nullable = true)
 |-- spi: boolean (nullable = true)
 |-- position_source: integer (nullable = true)



In [6]:
df.createOrReplaceTempView("EMP")

In [7]:
avia_c = spark_avia.sql('select origin_country, count( distinct icao24) as avia_amount from EMP group by origin_country order by avia_amount DESC')

In [31]:
avia_c .coalesce(1).write.csv("hdfs:///user/teran45/data/avia/country")

In [9]:
avia_c.show()

+--------------------+-----------+
|      origin_country|avia_amount|
+--------------------+-----------+
|       United States|      15769|
|           Australia|       1310|
|              Canada|       1112|
|             Germany|       1076|
|      United Kingdom|        943|
|              France|        580|
|               India|        568|
|              Turkey|        530|
|               Japan|        511|
|               China|        495|
|               Spain|        473|
|  Russian Federation|        467|
|United Arab Emirates|        463|
|             Ireland|        462|
|              Brazil|        406|
|Kingdom of the Ne...|        373|
|             Austria|        320|
|         Switzerland|        304|
|         New Zealand|        304|
|              Mexico|        295|
+--------------------+-----------+
only showing top 20 rows



In [11]:
avia_t = spark_avia.sql('select icao24, origin_country, velocity, to_timestamp(time_position) from EMP')

In [13]:
avia_t.show()

+------+--------------+--------+---------------------------+
|icao24|origin_country|velocity|to_timestamp(time_position)|
+------+--------------+--------+---------------------------+
|e49406|        Brazil|  242.57|                       null|
|a21b74| United States|   91.51|        2022-05-25 02:28:14|
|e8027b|         Chile|  234.62|        2022-05-25 02:29:23|
|a5a8e2| United States|  143.06|        2022-05-25 02:32:53|
|aa3cbe| United States|    5.66|        2022-05-25 02:32:52|
|c822af|   New Zealand|   82.62|        2022-05-25 02:32:52|
|ac494e| United States|   55.45|        2022-05-25 02:32:51|
|aa56db| United States|  235.94|        2022-05-25 02:32:45|
|7c6b2d|     Australia|  213.14|        2022-05-25 02:32:52|
|aa56d4| United States|     0.0|        2022-05-25 02:32:48|
|0d0c02|        Mexico|   244.5|        2022-05-25 02:32:53|
|a3687e| United States|  212.13|        2022-05-25 02:32:52|
|a5820f| United States|   32.74|        2022-05-25 02:30:26|
|aaef60| United States| 

In [14]:
avia_t.coalesce(1).write.csv("hdfs:///user/teran45/data/avia/speedtime")